In [ ]:
# PIP installs
!pip install -qU langchain_openai langchain_huggingface 
!pip install -qU langchain_core==0.2.40 langchain langchain_community langchain-text-splitters
!pip install -qU qdrant_client pymupdf tiktoken ragas pandas
!pip install -qU python-pptx==1.0.2 nltk==3.9.1

## Obtain the data from the PDF documents

In [1]:
import os, tiktoken
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Qdrant
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Path to my directory containing PDF files
directory = "References/"
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
# List to store all the documents
all_docs = []

# Iterate through all the files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".pdf"):  # Check if the file is a PDF
        file_path = os.path.join(directory, filename)
        loader = PyMuPDFLoader(file_path)
        docs = loader.load()
        all_docs.extend(docs)  # Append the loaded docs to my list


one_document = ""
for doc in all_docs:
    one_document += doc.page_content

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o").encode(
        text,
    )
    return len(tokens)

## I am splitting with different chunk and overlaps than are used in my app

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 50,
    length_function = tiktoken_len,
)

split_chunks = text_splitter.split_text(one_document)
training_documents = text_splitter.create_documents(split_chunks)
len(training_documents)

# training_vectorstore = Qdrant.from_documents(
#     text_splitter.create_documents(split_chunks),
#     embedding_model,
#     location=":memory:",
#     collection_name="training_collection",
# )

# training_retriever = training_vectorstore.as_retriever()

347

In [2]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

In [9]:
training_split_documents = training_documents[:250]
val_split_documents = training_documents[250:300]
test_split_documents = training_documents[300:347]
print(len(training_split_documents))
print(len(val_split_documents))
print(len(test_split_documents))
test_split_documents[46]

250
50
47


Document(metadata={'id': '6e590e51-727a-442b-b43b-24e90e0f935c'}, page_content='arXiv. https://arxiv.org/pdf/2309.01219 \nZhao, X. et al. (2023) Provable Robust Watermarking for AI-Generated Text. Semantic Scholar. \nhttps://www.semanticscholar.org/paper/Provable-Robust-Watermarking-for-AI-Generated-Text-Zhao-\nAnanth/75b68d0903af9d9f6e47ce3cf7e1a7d27ec811dc')

## Now build a fine tuning data set

In [10]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [11]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

In [12]:
question_generation_chain = qa_prompt_template | qa_chat_model

In [13]:
import tqdm

def create_questions(documents, n_questions):
  questions = {}
  relevant_docs = {}

  ### YOUR CODE HERE
  for document in tqdm.tqdm(documents):
    document_content = {"context": document.page_content, "questions": []}
    questions_generated = question_generation_chain.invoke({"context": document_content["context"], "n_questions": n_questions})
    for question in questions_generated.content.split("\n"):
      question_id = str(uuid.uuid4())
      questions[question_id] = "".join(question.split(".")[1:]).strip()
      relevant_docs[question_id] = [document.metadata["id"]]
  return questions, relevant_docs




  return questions, relevant_docs

In [14]:
## generate he data with n_questions = 2 for each
training_questions, training_relevant_contexts = create_questions(training_split_documents,2)
val_questions, val_relevant_contexts = create_questions(val_split_documents, 2) 
test_questions, test_relevant_contexts = create_questions(test_split_documents, 2)

100%|██████████| 47/47 [00:52<00:00,  1.12s/it]


### Save the datasets so I don't have to recreate them when I move into COLAB

In [15]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)

val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open("val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)

train_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : train_corpus
}

with open("test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)

## Now we will fine tune snowflake (what else)

In [16]:
!pip install -qU sentence_transformers datasets pyarrow

In [17]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-m"
model = SentenceTransformer(model_id)

/Users/jmichaeldean/anaconda3/envs/AIE4/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/84.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [18]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

In [19]:
BATCH_SIZE = 20

corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)


In [20]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [21]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [22]:
EPOCHS = 5

In [23]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned_arctic',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`